In [4]:
import pandas as pd
from pathlib import Path
base_dir = str(Path().resolve().parents[1])

In [9]:
for country in ['Egypt', 'Poland', 'Romania']:
    us10_path = 'US_10yr_yield.csv'
    us10 = pd.read_csv(us10_path)
    us10 = us10[2:][['Ticker', 'Series Type']]
    us10.rename(columns={'Ticker': 'Date', 'Series Type': 'ten_yr_us'}, inplace=True)
    us10['Date'] = pd.to_datetime(us10['Date'])
    us10['ten_yr_us'] = us10['ten_yr_us'].astype(float)

    if country == 'Romania':
        b10_path = base_dir + '\\Risk Premium\\Risk_Premium_' + country + '.csv'
        b10_name = "ten_yr_romania"
        b10 = pd.read_csv(b10_path)[["Date", b10_name]]
        b10['Date'] = pd.to_datetime(b10['Date']) + pd.tseries.offsets.MonthEnd(0)
    else:
        b10_path = base_dir + '\\Term Premium\\raw\\' + country + '_10yr_yield.csv'
        b10 = pd.read_csv(b10_path)
        b10 = b10[2:][['Ticker', 'Series Type']]
        b10_name = 'ten_yr_' + country.lower()
        b10.rename(columns={'Ticker': 'Date', 'Series Type': b10_name}, inplace=True)
        b10['Date'] = pd.to_datetime(b10['Date']) + pd.tseries.offsets.MonthEnd(0)
        b10[b10_name] = b10[b10_name].astype(float)

    term = pd.merge(b10, us10, how='inner')
    term['RiskPremium'] = term[b10_name] - term['ten_yr_us']
    csv_path = 'Risk_Premium_' + country + '.csv'
    term.to_csv(csv_path, index=False)
    